In [1]:
import os
os.chdir("Desktop")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Desktop'

In [56]:
import numpy as np
import pandas as pd
import re
import collections
from nltk import word_tokenize as wt
from nltk import pos_tag as pt
from nltk.corpus import stopwords
from stanfordcorenlp import StanfordCoreNLP
from nltk import pos_tag
import gensim

C:\Users\santh_000\Anaconda3\envs\tfp3.6\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [44]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import GRU
from keras.layers import MaxoutDense
from keras.layers import Activation,Flatten,Dense
from keras.utils import to_categorical
from keras.layers import LSTM


In [4]:
data = pd.read_csv("restaurant.csv", sep=',')

In [5]:
data.head()

,example_id,text,aspect_term,term_location,class
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0


In [6]:
data['text'] = data['text'].apply(lambda x: x.replace("[comma]", ',').replace("_", " ").strip())
data['aspect_term'] = data['aspect_term'].apply(lambda x: x.replace("[comma]", ',').replace("_", " ").strip())
data['text'] = data['text'].apply(lambda x: re.sub('\W+',' ', x))
data['aspect_term'] = data['aspect_term'].apply(lambda x: re.sub('\W+',' ', x))
stop = set(stopwords.words('english'))
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data['aspect_term'] = data['aspect_term'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data['text'] = data['text'].str.lower()
data['aspect_term'] = data['aspect_term'].str.lower()

In [7]:
data.head()

,example_id,text,aspect_term,term_location,class
0,3121_0,but staff horrible us,staff,8--13,-1
1,2777_0,to completely fair redeeming factor food avera...,food,57--61,1
2,1634_0,the food uniformly exceptional capable kitchen...,food,4--8,1
3,1634_1,the food uniformly exceptional capable kitchen...,kitchen,55--62,1
4,1634_2,the food uniformly exceptional capable kitchen...,menu,141--145,0


In [8]:
pos = []
for sent in data['text']:
    sent=sent.split()
    pos.append(pos_tag(sent))

In [9]:
pos[:1]

[[('but', 'CC'), ('staff', 'NN'), ('horrible', 'JJ'), ('us', 'PRP')]]

In [10]:
pos_matrix = []
for sent in pos:
    pos_matrix.append([x for x in sent if re.match('\W+', x[0]) is None])

In [12]:
feat_mat = []
pos_feat_mat = []
word_mat = []
progress = 0
prog = 0
for row,aspect in zip(pos_matrix,data['aspect_term']):
    prog += 1
    skip_row = False
    left_dist = []
    right_dist = []
    left_pos = []
    right_pos = []
    left_word = []
    right_word = []
    aspect_len = len(aspect.split())
    split_aspect = pt(wt(aspect))
    for i in range(len(row)):

        if row[i][0] == split_aspect[0][0]: 
        
            check = [row[q][0] for q in range(i,i+aspect_len)]
            
            if collections.Counter(check) == collections.Counter(aspect.split()):
                skip_row = True 
                
                window = 10 - aspect_len
                left = window // 2 
                right = window - left 
                
                l = i-left 
                r = i+aspect_len-1+right 
                
                
                if l < 0 and r > len(row)-1: 
                    left += l
                    right -= r-len(row)+1
                
                elif l < 0: 
                    if r-l <= len(row)-1:
                        right += (-1)*l 
                    else:
                        right += len(row) - 1 - r
                    left -= (-1)*l
                        
                elif r > len(row)-1: 
                    if l-(r-len(row)+1) >= 0:
                        left += r-len(row)+1 #take words from left
                    else: 
                        left += l
                    right -= r-len(row)+1
                        
               
                
                count = 0
                j = 1
                    
                while count < left:
                    if row[i-j][0][0].isalnum():
                        count += 1
                        left_dist.append(-1*count)
                        left_pos.append(row[i-j][1])
                        left_word.append(row[i-j][0])
                    j += 1
                    
                count = 0
               
                j = aspect_len
            
                while count < right:
                    if row[i+j][0][0].isalnum() and row[i+j][0] not in split_aspect[0][0]:
                        count += 1
                      
                        right_dist.append(count)
                        right_pos.append(row[i+j][1])
                        right_word.append(row[i+j][0])

                    if row[i+j][0] in split_aspect[0][0]: #if first word of aspect and current word matches
                        if i+j+aspect_len-1 <= len(row)-1: 
                            check1 = [row[q][0] for q in range(i+j,i+j+aspect_len-1)]
                            if collections.Counter(check1) == collections.Counter(aspect.split()):
                                j += aspect_len-1
                            else:
                                count += 1
                                right_dist.append(count)
                                right_pos.append(row[i+j][1])
                                right_word.append(row[i+j][0])
                    
                        else: 
                            count += 1
                            right_dist.append(count)
                            right_pos.append(row[i+j][1])
                            right_word.append(row[i+j][0])
                                
                             
                    j += 1
                progress += 1
                break
                
                
                
    pos_aspect = []
    for p in range(len(aspect.split())): 
        try:
            pos_aspect.append(row[i+p][1])
        except IndexError:
            pass
            continue
        
    
    if skip_row == False:
        print("{} row not considered in traversal".format(prog-1))
    
    rem = 10 - (aspect_len + left + right)
    
    if aspect_len <= 10:
        feat_mat.append([0]*rem + left_dist[::-1] + [0]*len(aspect.split()) + right_dist)
        pos_feat_mat.append(['NULL']*rem + left_pos[::-1] + pos_aspect + right_pos)
        word_mat.append([0]*rem + left_word[::-1] + aspect.split() + right_word)
        
    else: 
        feat_mat.append([0]*rem + left_dist[::-1] + [0]*10 + right_dist)
        pos_feat_mat.append(['NULL']*rem + left_pos[::-1] + pos_aspect[:10] + right_pos)
        word_mat.append([0]*rem + left_word[::-1] + aspect.split()[:10] + right_word)

3260 row not considered in traversal
3524 row not considered in traversal


In [13]:
feat_mat

[[0, 0, 0, 0, 0, 0, -1, 0, 1, 2],
 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4],
 [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
 [-9, -8, -7, -6, -5, -4, -3, -2, -1, 0],
 [0, 0, 0, 0, -1, 0, 1, 2, 3, 4],
 [0, 0, 0, 0, -4, -3, -2, -1, 0, 1],
 [-3, -2, -1, 0, 0, 0, 1, 2, 3, 4],
 [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
 [-9, -8, -7, -6, -5, -4, -3, -2, -1, 0],
 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4],
 [0, -1, 0, 1, 2, 3, 4, 5, 6, 7],
 [0, 0, 0, 0, 0, 0, -1, 0, 1, 2],
 [-3, -2, -1, 0, 1, 2, 3, 4, 5, 6],
 [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4],
 [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
 [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
 [-8, -7, -6, -5, -4, -3, -2, -1, 0, 1],
 [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
 [-8, -7, -6, -5, -4, -3, -2, -1, 0, 1],
 [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
 [0, 0, 0, 0, 0, 0, -1, 0, 1, 2],
 [0, 0, 0, 0, 0, 0, -2, -1, 0, 1],
 [0, 0, 0, 0, -5, -4, -3, -2, -1, 0],
 [0, 0, 0, -1, 0, 1, 2, 3, 4, 5],
 [0, 0, 0, -4, -3, -2, -1, 0, 0, 0],
 [0, 0,

In [14]:
pos_feat_mat

[['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'CC', 'NN', 'JJ', 'PRP'],
 ['TO', 'RB', 'JJ', 'VBG', 'NN', 'NN', 'JJ', 'NN', 'NNS', 'VBP'],
 ['DT', 'NN', 'JJ', 'JJ', 'JJ', 'NN', 'RB', 'VBD', 'WDT', 'VBP'],
 ['NN', 'JJ', 'JJ', 'JJ', 'NN', 'RB', 'VBD', 'WDT', 'VBP', 'IN'],
 ['JJ', 'NN', 'RB', 'VBD', 'WDT', 'VBP', 'IN', 'VBG', 'IN', 'NN'],
 ['NULL', 'NULL', 'NULL', 'NULL', 'RB', 'NN', 'JJ', 'JJ', 'NNS', 'JJ'],
 ['NULL', 'NULL', 'NULL', 'NULL', 'RB', 'NN', 'JJ', 'JJ', 'NNS', 'JJ'],
 ['PRP$', 'VBD', 'JJ', 'JJ', 'NN', 'VBN', 'RB', 'NNS', 'NN', 'JJ'],
 ['JJ', 'NN', 'VBN', 'RB', 'NNS', 'NN', 'JJ', 'NN', 'VB', 'NN'],
 ['RB', 'NNS', 'NN', 'JJ', 'NN', 'VB', 'NN', 'VB', 'JJ', 'NNS'],
 ['RB', 'NNS', 'NN', 'JJ', 'NN', 'VB', 'NN', 'VB', 'JJ', 'NNS'],
 ['NULL', 'DT', 'NNS', 'JJ', 'NN', 'JJ', 'NN', 'NN', 'RB', 'JJ'],
 ['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'RB', 'NN', 'RB', 'JJ'],
 ['PRP', 'VBP', 'JJ', 'NN', 'VBD', 'NNS', 'VB', 'JJ', 'NN', 'JJ'],
 ['PRP', 'VBP', 'JJ', 'NN', 'VBD', 'NNS', 'V

In [15]:
word_mat

[[0, 0, 0, 0, 0, 0, 'but', 'staff', 'horrible', 'us'],
 ['to',
  'completely',
  'fair',
  'redeeming',
  'factor',
  'food',
  'average',
  'make',
  'deficiencies',
  'teodora'],
 ['the',
  'food',
  'uniformly',
  'exceptional',
  'capable',
  'kitchen',
  'proudly',
  'whip',
  'whatever',
  'feel'],
 ['food',
  'uniformly',
  'exceptional',
  'capable',
  'kitchen',
  'proudly',
  'whip',
  'whatever',
  'feel',
  'like'],
 ['capable',
  'kitchen',
  'proudly',
  'whip',
  'whatever',
  'feel',
  'like',
  'eating',
  'whether',
  'menu'],
 [0, 0, 0, 0, 'not', 'food', 'outstanding', 'little', 'perks', 'great'],
 [0, 0, 0, 0, 'not', 'food', 'outstanding', 'little', 'perks', 'great'],
 ['our',
  'agreed',
  'favorite',
  'orrechiete',
  'sausage',
  'chicken',
  'usually',
  'waiters',
  'kind',
  'enough'],
 ['orrechiete',
  'sausage',
  'chicken',
  'usually',
  'waiters',
  'kind',
  'enough',
  'split',
  'dish',
  'half'],
 ['usually',
  'waiters',
  'kind',
  'enough',
  'spli

In [40]:
t = Tokenizer()
t.fit_on_texts(feat_mat)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(feat_mat)



In [41]:
encoded_docs

[[1, 1, 1, 1, 1, 1, 2, 1, 3, 5],
 [11, 8, 6, 4, 2, 1, 3, 5, 7, 9],
 [2, 1, 3, 5, 7, 9, 10, 13, 15, 17],
 [8, 6, 4, 2, 1, 3, 5, 7, 9, 10],
 [18, 16, 14, 12, 11, 8, 6, 4, 2, 1],
 [1, 1, 1, 1, 2, 1, 3, 5, 7, 9],
 [1, 1, 1, 1, 8, 6, 4, 2, 1, 3],
 [6, 4, 2, 1, 1, 1, 3, 5, 7, 9],
 [8, 6, 4, 2, 1, 3, 5, 7, 9, 10],
 [18, 16, 14, 12, 11, 8, 6, 4, 2, 1],
 [11, 8, 6, 4, 2, 1, 3, 5, 7, 9],
 [1, 2, 1, 3, 5, 7, 9, 10, 13, 15],
 [1, 1, 1, 1, 1, 1, 2, 1, 3, 5],
 [6, 4, 2, 1, 3, 5, 7, 9, 10, 13],
 [2, 1, 3, 5, 7, 9, 10, 13, 15, 17],
 [11, 8, 6, 4, 2, 1, 3, 5, 7, 9],
 [8, 6, 4, 2, 1, 3, 5, 7, 9, 10],
 [8, 6, 4, 2, 1, 3, 5, 7, 9, 10],
 [16, 14, 12, 11, 8, 6, 4, 2, 1, 3],
 [8, 6, 4, 2, 1, 3, 5, 7, 9, 10],
 [16, 14, 12, 11, 8, 6, 4, 2, 1, 3],
 [8, 6, 4, 2, 1, 3, 5, 7, 9, 10],
 [1, 1, 1, 1, 1, 1, 2, 1, 3, 5],
 [1, 1, 1, 1, 1, 1, 4, 2, 1, 3],
 [1, 1, 1, 1, 11, 8, 6, 4, 2, 1],
 [1, 1, 1, 2, 1, 3, 5, 7, 9, 10],
 [1, 1, 1, 8, 6, 4, 2, 1, 1, 1],
 [1, 1, 6, 4, 2, 1, 1, 3, 5, 7],
 [1, 1, 14, 12, 11, 8, 6, 4, 2, 1]